<a href="https://colab.research.google.com/github/AbdulWahabRaza123/ML/blob/main/PipelinesOptimization(Prefetch_and_Cache).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.9.2'

In [23]:
class FileDataset(tf.data.Dataset):
  def read_files_in_batches(num_samples):
    #open file (assuming everything here)
    time.sleep(0.03)
    for sample_idx in range(num_samples):
      time.sleep(0.015)
      yield (sample_idx,)
  def __new__(cls,num_samples=3):
    print("New Methos is Called...")
    #Here cls is class object as parameter
    return tf.data.Dataset.from_generator(
        cls.read_files_in_batches,
        output_signature=tf.TensorSpec(shape=(1,),dtype=tf.int64),
        args=(num_samples,)
    )

In [24]:
def benchmark(dataset,num_epochs=2):
  for epochs_num in range(num_epochs):
    for sample in dataset:
      time.sleep(0.01)

In [29]:
%%timeit
benchmark(FileDataset())

New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
297 ms ± 16.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
#by using prefetech API we will get that time is reduced
%%timeit
benchmark(FileDataset().prefetch(1))

New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
238 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
#by using prefetech API we will get that time is more reduced using autotune parameter (More Preferable)
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
New Methos is Called...
276 ms ± 41.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


###Cache API

In [34]:
dataset=tf.data.Dataset.range(5)
for d in dataset:
  print(d.numpy())

0
1
2
3
4


In [35]:
#creating square of these numbers
dataset=dataset.map(lambda x:x**2)
for d in dataset:
  print(d.numpy())

0
1
4
9
16


In [50]:
dataset=dataset.cache()
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [51]:
def mapped_function(s):
  tf.py_function(lambda: time.sleep(0.03),[],())
  return s

In [52]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function),5)

New Methos is Called...
1.51 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [53]:
#let's use chache
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function).cache(),5)

New Methos is Called...
416 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
